In [ ]:
import openml
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

from automl import AutoML

[W 2025-11-14 18:52:11,284] Trial 0 failed with parameters: {'max_iter': 8277, 'fit_intercept': True, 'C': 1.8590843630169627, 'solver': 'saga', 'penalty': 'elasticnet', 'l1_ratio': 0.9636627605010293} because of the following error: ValueError('Input X contains NaN.\nLogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values').
Traceback (most recent call last):
  File "c:\Users\Aleksandr\Documents\python_apps\MAS_AutoML\.venv\Lib\site

In [1]:
import openml

In [2]:

task = openml.tasks.get_task(31)
class_labels = task.class_labels


In [3]:
class_labels

['bad', 'good']

In [2]:
import openml
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

from automl import AutoML


# Инициализация переменных
task_id = 168868

task = None
dataset = None
X = None
y = None
categorical = None
feature_names = None
n_folds = 3
class_labels = None
label_mapping = None
data_loaded = False

print("[INFO] Начало загрузки данных из OpenML...")

try:
    print(f"[INFO] Загрузка задачи из OpenML (task_id={task_id})...")
    task = openml.tasks.get_task(task_id)
    print(f"[INFO] Задача загружена: {task.task_id}")
    
    print("[INFO] Загрузка датасета...")
    dataset = task.get_dataset()
    print(f"[INFO] Датасет загружен: {dataset.name}")
    
    print("[INFO] Получение параметров задачи...")
    params = task.estimation_parameters
    n_folds = int(params.get("number_folds", 3))
    print(f"[INFO] Количество фолдов: {n_folds}")
    
    class_labels = task.class_labels
    print(f"[INFO] Классы: {class_labels}")
    
    print("[INFO] Загрузка данных...")
    X, y, categorical, feature_names = dataset.get_data(target=dataset.default_target_attribute)
    print(f"[INFO] Данные загружены. Размер X: {X.shape}, размер y: {y.shape}")
    
    # Проверки данных
    if X is None:
        print("[ERROR] X равен None. Пропуск дальнейшей обработки.")
    elif isinstance(X, pd.DataFrame) and X.empty:
        print("[ERROR] X пустой DataFrame. Пропуск дальнейшей обработки.")
    elif isinstance(X, np.ndarray) and X.size == 0:
        print("[ERROR] X пустой ndarray. Пропуск дальнейшей обработки.")
    elif len(X) == 0:
        print("[ERROR] X имеет нулевую длину. Пропуск дальнейшей обработки.")
    elif y is None:
        print("[ERROR] y равен None. Пропуск дальнейшей обработки.")
    elif isinstance(y, pd.Series) and y.empty:
        print("[ERROR] y пустой Series. Пропуск дальнейшей обработки.")
    elif isinstance(y, (pd.Series, np.ndarray)) and len(y) == 0:
        print("[ERROR] y имеет нулевую длину. Пропуск дальнейшей обработки.")
    elif len(X) != len(y):
        print(f"[ERROR] Несоответствие размеров: X={len(X)}, y={len(y)}. Пропуск дальнейшей обработки.")
    else:
        print("[INFO] Создание маппинга меток...")
        label_mapping = {v: k for k, v in enumerate(class_labels)}
        if isinstance(y, pd.Series):
            y = np.array(y.map(label_mapping), dtype=int)
        else:
            # Если y уже массив или другой тип, преобразуем через pandas для совместимости
            y_series = pd.Series(y)
            y = np.array(y_series.map(label_mapping), dtype=int)
        
        # Проверка после преобразования
        if len(y) == 0:
            print("[ERROR] y пустой после преобразования. Пропуск дальнейшей обработки.")
        elif len(np.unique(y)) < 2:
            print(f"[ERROR] Недостаточно классов в y: {np.unique(y)}. Пропуск дальнейшей обработки.")
        else:
            print(f"[INFO] Уникальные классы в y: {np.unique(y)}")
            print(f"[INFO] Распределение классов: {pd.Series(y).value_counts().to_dict()}")
            
            data_loaded = True
            print("[INFO] Данные успешно загружены и подготовлены")
    
except Exception as e:
    print(f"[ERROR] Ошибка при загрузке данных: {type(e).__name__}: {e}")
    data_loaded = False
    print("[WARNING] Пайплайн остановлен из-за ошибки загрузки данных")

[INFO] Начало загрузки данных из OpenML...
[INFO] Загрузка задачи из OpenML (task_id=168868)...
[INFO] Задача загружена: 168868
[INFO] Загрузка датасета...
[INFO] Датасет загружен: APSFailure
[INFO] Получение параметров задачи...
[INFO] Количество фолдов: 10
[INFO] Классы: ['neg', 'pos']
[INFO] Загрузка данных...
[INFO] Данные загружены. Размер X: (76000, 170), размер y: (76000,)
[INFO] Создание маппинга меток...
[INFO] Уникальные классы в y: [0 1]
[INFO] Распределение классов: {0: 74625, 1: 1375}
[INFO] Данные успешно загружены и подготовлены


In [5]:
automl = None
model_trained = False

if not data_loaded:
    print("[ERROR] Данные не загружены. Пропуск обучения модели.")
else:
    try:
        # Проверки перед обучением
        if X is None or y is None:
            print("[ERROR] X или y не определены. Пропуск обучения модели.")
        elif not isinstance(X, (pd.DataFrame, np.ndarray)):
            print(f"[ERROR] X должен быть DataFrame или ndarray, получен {type(X)}. Пропуск обучения модели.")
        elif not isinstance(y, np.ndarray):
            print(f"[ERROR] y должен быть ndarray, получен {type(y)}. Пропуск обучения модели.")
        else:
            print("[INFO] Инициализация AutoML...")
            print(f"[INFO] Параметры: task=classification, n_splits={n_folds}, n_jobs=3")
            
            automl = AutoML(
                task='classification',
                use_preprocessing_pipeline=True,
                feature_selector_type=None,
                use_val_test_pipeline=False,
                auto_models_init_kwargs = {
                    "metric": "roc_auc",
                    "time_series": False,
                    "models_list": ["linear", "forests", "boostings"],
                    "blend": True,
                    "stack": True,
                    "n_splits": n_folds,
                },
                n_jobs=12, 
                random_state=0,
            )
            print("[INFO] AutoML инициализирован")
            
            print("[INFO] Начало обучения модели...")
            automl = automl.fit(
                X, y, 
                auto_model_fit_kwargs = {
                    "tuning_timeout": 10
                }
            )
            
            # Проверка результата обучения
            if automl is None:
                print("[ERROR] AutoML вернул None после обучения. Пропуск дальнейшей обработки.")
            elif not hasattr(automl, 'auto_model'):
                print("[ERROR] AutoML не имеет атрибута auto_model. Пропуск дальнейшей обработки.")
            else:
                model_trained = True
                print("[INFO] Модель успешно обучена")
                
                if hasattr(automl.auto_model, 'best_score'):
                    print(f"[INFO] Лучший OOF score: {automl.auto_model.best_score}")
        
    except Exception as e:
        print(f"[ERROR] Ошибка при обучении модели: {type(e).__name__}: {e}")
        import traceback
        print(f"[ERROR] Трассировка: {traceback.format_exc()}")
        model_trained = False
        print("[WARNING] Модель не обучена")

[INFO] Инициализация AutoML...
[INFO] Параметры: task=classification, n_splits=10, n_jobs=3
[2025-11-14 18:52:54,050] - [  PREPROC   ] - Успешно заданы шаги pipeline
[INFO] AutoML инициализирован
[INFO] Начало обучения модели...
[2025-11-14 18:52:54,074] - [  PREPROC   ] - NaN features to drop: ['ab_000', 'ad_000', 'bk_000', 'bl_000', 'bm_000', 'bn_000', 'bo_000', 'bp_000', 'bq_000', 'br_000', 'cf_000', 'cg_000', 'ch_000', 'co_000', 'cr_000', 'ct_000', 'cu_000', 'cv_000', 'cx_000', 'cy_000', 'cz_000', 'da_000', 'db_000', 'dc_000']
[2025-11-14 18:52:54,144] - [Pipeline] .. (step 1 of 6) Processing nan_cols_dropper, total=   0.1
[2025-11-14 18:52:55,756] - [Pipeline] ....... (step 2 of 6) Processing nan_imputer, total=   1.6
[2025-11-14 18:52:56,163] - [  PREPROC   ] - QConstant features to drop: ['ae_000', 'af_000', 'ag_000', 'ag_001', 'ak_000', 'ar_000', 'as_000', 'au_000', 'ay_000', 'ay_001', 'ay_002', 'ay_003', 'ay_004', 'ay_009', 'az_009', 'cd_000', 'cn_000', 'cs_009', 'df_000', 'dg

In [12]:
import pandas as pd
import numpy as np
from pathlib import Path
import json
from datetime import datetime
from automl import AutoML


CSV_PATH_TO_PREDICT = 'C:\\Users\\User1\\Desktop\\ITMO_bootcamp\\data\\datasets\\TEST\\test1__.csv'
class_labels =  ['good', 'bad']
# Загружаем данные
csv_path = 'C:\\Users\\User1\\Desktop\\ITMO_bootcamp\\data\\datasets\\openml_31_credit-g.csv'
df = pd.read_csv('C:\\Users\\User1\\Desktop\\ITMO_bootcamp\\data\\datasets\\openml_31_credit-g.csv')
print(f"Загружено строк: {{len(df)}}")
label = 'class'
# Генерированный


X = df.drop(columns=[label])
y = df[label]
import pandas as pd
import numpy as np

label_mapping = {v: k for k, v in enumerate(class_labels)}

if isinstance(y, pd.Series):
    y = np.array(y.map(label_mapping), dtype=int)
else:
    y_series = pd.Series(y)
    y = np.array(y_series.map(label_mapping), dtype=int)


automl = AutoML(
    task='classification',
    use_preprocessing_pipeline=False,
    feature_selector_type=None,
    use_val_test_pipeline=False,
    auto_models_init_kwargs={
        "metric": "roc_auc",
        "time_series": False,
        "models_list": ["linear", "forests", "boostings"],
        "blend": True,
        "stack": True,
        "n_splits": 10
    },
    n_jobs=3,
    random_state=0,
)

automl = automl.fit(
    X,
    y,
    auto_model_fit_kwargs={"tuning_timeout": 10}
)

score = automl.auto_model.best_score
preds = automl.predict(df)
test_predictions = preds[:, 1]


Загружено строк: {len(df)}
[2025-11-15 01:12:31,988] - [   MODEL    ] - 1 out of 8. type
[2025-11-15 01:12:31,989] - [   START    ] - Working with type
[2025-11-15 01:12:31,990] - [   START    ] - Tuning type
[2025-11-15 01:12:38,251] - [   OPTUNA   ] - Trial 0. New best score 0.6021904761904763 with parameters {'max_iter': 8277, 'fit_intercept': True, 'C': 1.8590843630169627, 'solver': 'saga', 'penalty': 'elasticnet', 'l1_ratio': 0.9636627605010293}
[2025-11-15 01:12:42,758] - [   OPTUNA   ] - Trial 1. New best score 0.6021904761904763 with parameters {'max_iter': 5813, 'fit_intercept': True, 'C': 2.5601615250028704, 'solver': 'saga', 'penalty': None}
[2025-11-15 01:12:42,759] - [   OPTUNA   ] - 2 trials completed
[2025-11-15 01:12:42,760] - [BEST PARAMS ] - {'random_state': 0, 'n_jobs': 3, 'verbose': 0, 'max_iter': 8277, 'fit_intercept': True, 'C': 1.8590843630169627, 'solver': 'saga', 'penalty': 'elasticnet', 'l1_ratio': 0.9636627605010293, 'time_series': False, 'model_type': 'class

ValueError: X has 21 features, but RandomForestClassifier is expecting 20 features as input.

In [6]:
oof_score = None
score = None
predictions = None

if not model_trained or automl is None:
    print("[ERROR] Модель не обучена. Пропуск предсказаний и оценки.")
else:
    try:
        print("[INFO] Получение предсказаний...")
        
        # Проверки перед предсказанием
        if not hasattr(automl, 'predict'):
            print("[ERROR] AutoML не имеет метода predict. Пропуск предсказаний.")
        elif X is None:
            print("[ERROR] X не определен. Пропуск предсказаний.")
        else:
            predictions = automl.predict(X)
            
            if predictions is None:
                print("[ERROR] Предсказания равны None. Пропуск дальнейшей обработки.")
            elif len(predictions) == 0:
                print("[ERROR] Предсказания пустые. Пропуск дальнейшей обработки.")
            else:
                print(f"[INFO] Получены предсказания. Размер: {predictions.shape}")
                
                # Проверка формы предсказаний для бинарной классификации
                if len(predictions.shape) == 2 and predictions.shape[1] >= 2:
                    predictions_proba = predictions[:, 1]
                    print("[INFO] Используется вероятность второго класса")
                elif len(predictions.shape) == 1:
                    predictions_proba = predictions
                    print("[INFO] Используются предсказания напрямую")
                else:
                    print(f"[ERROR] Неожиданная форма предсказаний: {predictions.shape}. Пропуск дальнейшей обработки.")
                    predictions_proba = None
                
                if predictions_proba is not None:
                    # Получение OOF score
                    if hasattr(automl.auto_model, 'best_score'):
                        oof_score = automl.auto_model.best_score
                        print(f"[INFO] OOF score: {oof_score}")
                    else:
                        print("[WARNING] best_score не найден в auto_model")
                    
                    # Вычисление метрики на полных данных
                    print("[INFO] Вычисление ROC-AUC на полных данных...")
                    if y is None:
                        print("[ERROR] y не определен. Пропуск вычисления метрики.")
                    elif len(y) != len(predictions_proba):
                        print(f"[ERROR] Несоответствие размеров: y={len(y)}, predictions={len(predictions_proba)}. Пропуск вычисления метрики.")
                    else:
                        try:
                            score = roc_auc_score(y, predictions_proba)
                            print(f"[INFO] ROC-AUC на полных данных: {score}")
                            print(f"[INFO] Результаты: OOF={oof_score}, Full={score}")
                        except Exception as metric_error:
                            print(f"[ERROR] Ошибка при вычислении ROC-AUC: {type(metric_error).__name__}: {metric_error}")
                            print("[WARNING] Метрика не вычислена")
        
    except Exception as e:
        print(f"[ERROR] Ошибка при получении предсказаний или оценке: {type(e).__name__}: {e}")
        import traceback
        print(f"[ERROR] Трассировка: {traceback.format_exc()}")
        print("[WARNING] Предсказания или оценка не выполнены")

# Вывод результатов
if oof_score is not None and score is not None:
    print(f"\n[RESULT] OOF score: {oof_score}, Full data score: {score}")
    (oof_score, score)
else:
    print("\n[RESULT] Результаты недоступны")
    None

[INFO] Получение предсказаний...
[INFO] Получены предсказания. Размер: (76000, 2)
[INFO] Используется вероятность второго класса
[INFO] OOF score: 0.9923252042028323
[INFO] Вычисление ROC-AUC на полных данных...
[INFO] ROC-AUC на полных данных: 0.9998607632099894
[INFO] Результаты: OOF=0.9923252042028323, Full=0.9998607632099894

[RESULT] OOF score: 0.9923252042028323, Full data score: 0.9998607632099894
